In [ ]:
!pip install pandas_datareader
!pip install seaborn
!pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import datetime
import matplotlib as mpl
from scipy.optimize import minimize
import yfinance as yf
yf.pdr_override()
PATH='data/'

In [ ]:
def get_portfolio(ticker,df):
    result = df[df['ticker']==ticker]['final_stocks'].values
    if len(result)>0:
        res = []
        for data in result[0]:
            ticker,exch = data.split()
            if exch =='US':
                res.append(ticker)
            else:
                res.append(ticker+'.'+exch)
        return res

    else:
        print("해당 종목은 찾을 수 없는 종목입니다")
        return None

In [ ]:
No_ISIN_list =[]
def find_incode(stock):
    result=  stock_name_data[stock_name_data[0]==stock][1].values
    if len(result)>0:
        return result[0]
    else:
        return stock
def make_price_score(ticker):
    try :
        new_dict=price_corr_df[ticker].to_dict()
        sorted_dict = {key: value for key, value in sorted(new_dict.items(), key=lambda item: item[1],reverse=True)}
        top_items = {key: sorted_dict[key] for key in list(sorted_dict.keys())[1:11]}
        bottom_items = {key: sorted_dict[key] for key in list(sorted_dict.keys())[-10:]}
        merged_dict = top_items.copy()  # 첫 번째 딕셔너리를 복사하여 새로운 딕셔너리 생성
        merged_dict.update(bottom_items)
        return {find_ISIN_with_ticker(key): value for key, value in merged_dict.items()}
    except Exception as E:
        print(E)
        return{}

def find_ISIN_with_ticker(ticker):
    result = df[df['ticker']==ticker]['ISIN'].values
    if len(result)>0:
        return result[0]
    else:
        return ticker

def make_price_ticker(ticker):
    a,b = ticker.split(' ')
    ticker,exch = ticker.split()
    if exch =='US':
        return ticker
    else:
        return (ticker+'.'+exch)

#해당 score를 normalize 합니다.
def normalize(value,min_value,max_value):
    return (value - min_value) / (max_value - min_value) if max_value != min_value else 1

#해당 score_dcit을 normalize 합니다.
def normalize_dict(original):
    if not original:
        return {}
    max_value = max(original.values())
    min_value = min(original.values())
    if min_value<0:
        normalized_dict = {key: -normalize(value,max_value,min_value) for key, value in original.items()}
        sorted_dict = {key: value for key, value in sorted(normalized_dict.items(), key=lambda item: item[1],reverse=False)}

    else:
        normalized_dict = {key: normalize(value,min_value,max_value) for key, value in original.items()}
        sorted_dict = {key: value for key, value in sorted(normalized_dict.items(), key=lambda item: item[1],reverse=True)}


    return sorted_dict

#해당 각 관계의 dcit을 normalize 합니다.
def merge_dict(dick_list):
    merged_dict = {}
    dict_list= [normalize_dict(x) for x in dick_list]
    for x in dict_list:
        for key, value in x.items():
            merged_dict[key] = merged_dict.get(key, 0) + value
    sorted_dict = {key: value for key, value in sorted(merged_dict.items(), key=lambda item: item[1],reverse=True)}
    return sorted_dict

#해당 각 상위와 하위위 dcit 후보를 normalize 합니다.
def make_stock_list(merged_dict,top, bottom):
    top_items = {key: merged_dict[key] for key in list(merged_dict.keys())[:top]}
    if bottom == 0:
        bottom_items = {}
    else:
        bottom_items = {key: merged_dict[key] for key in list(merged_dict.keys())[-bottom:]}
    merged_dict = merge_dict([top_items,bottom_items])
    return merged_dict

#ISIN을 기반으로 ticker를 가져옵니다.
def find_ticker(ISIN):
    ticker=  domain_df[domain_df['ISIN']==ISIN]['ticker'].values
    exchCode = domain_df[domain_df['ISIN']==ISIN]['exchCode'].values
    if len(ticker)>0:
        if len(exchCode)>0:
            return ticker[0]+' '+exchCode[0]
    else:
        ticker=  ISIN_ticker_data[ISIN_ticker_data['ISIN']==ISIN]['ticker'].values
        if len(ticker)>0:
            return ticker[0]
        return ISIN

#dedicate된 후보들을 tiker화 시켜 반환합니다.
def make_stock_list_with_ticker(final_dict,input_stock):
    stocklist =list(final_dict.keys())
    stocklist.append(input_stock)
    if not stocklist:
        return None
    else:
        stocklist= [ find_ticker(x) for x in stocklist]
        return stocklist

#관계 scoere를 기반으로 가져올 연관비중을 선택하여 최종 포트폴리우 구성 후보를 가져옵니다.
def get_portfolio_ratio(ticker,df,top,bottom):
    df['dedicate_stocks'] =None
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Calculating scores"):
        df.at[index, 'dedicate_stocks'] = make_stock_list(row['total_score'],top, bottom)
    df['final_stocks'] =None
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Calculating scores"):
        df.at[index, 'final_stocks'] = make_stock_list_with_ticker(row['dedicate_stocks'],row['ISIN'])
    result = df[df['ticker']==ticker]['final_stocks'].values
    if len(result)>0:
        res = []
        for data in result[0]:
            ticker,exch = data.split()
            if exch =='US':
                res.append(ticker)
            else:
                res.append(ticker+'.'+exch)
        return res

    else:
        print("해당 종목은 찾을 수 었는 종목입니다")
        return None

In [ ]:
domain_df = pd.read_csv(f"{PATH}domain_data.csv")
ISIN_ticker_data= pd.read_pickle(f"{PATH}ISIN_ticker_data.pkl")
price_corr_df = pd.read_pickle(f"{PATH}stock_price_corr_data")
relation_score = pd.read_pickle(f"{PATH}domain_total_filter.pkl")

# 학습 데이터 셋 설정

In [ ]:
# 학습 데이터셋 기간 설정
start = datetime.datetime(2023, 1, 25)
end = datetime.datetime(2023, 8, 30)

In [ ]:
# 상관관계 정도 상위 5개 하위 4개 종목
tickers = get_portfolio_ratio('AMZN',relation_score, 4, 5)

In [ ]:
tickers

In [ ]:
num_kr = pd.Series(tickers).str.endswith(('.KS', '.KQ')).sum()
num_us = (~pd.Series(tickers).str.endswith(('.KS', '.KQ'))).sum()
adjClose = pd.DataFrame()
#exch_list =['.KS','.KQ']

for item in tickers:
        data = web.get_data_yahoo(item, start=start, end=end)
        if len(data)<1:
            a, b = item.split('.')
            if b == 'KS':
                item = a + '.KQ'
                data = web.get_data_yahoo(item, start=start, end=end)
                adjClose[item] = data['Adj Close'].astype(float)
            if b == 'KQ':
                item = a + '.KS'
                data = web.get_data_yahoo(item, start=start, end=end)
                adjClose[item] = data['Adj Close'].astype(float)

        adjClose[item] = data['Adj Close'].astype(float)

# 로그 리턴 계산
daily_log_returns = np.log(adjClose/adjClose.shift(1)).dropna()
# 연간 로그 수익률 계산
meanReturns = daily_log_returns.mean()
annualReturns = daily_log_returns.mean() * 252
# 종목간의 수익률 공분산 계산
cov_daily = daily_log_returns.cov()
cov_annual = cov_daily * 252


In [ ]:
adjClose.columns

In [ ]:
num_kr

In [ ]:
num_us

In [ ]:
# KOFR = 한국 무위험 수익률 출처:(https://www.kofr.kr/main.jsp)
rf1 = pd.read_csv(f"{PATH}KOFR.csv")
kor_rf = round((rf1['KOFR'].mean()) / 100 , 4)
kor_rf

In [ ]:
# SOFR = 미국 무위험 수익률 출처:(https://www.newyorkfed.org/markets/reference-rates/sofr)
rf2 = pd.read_csv(f"{PATH}SOFR.csv")
us_rf = round((rf2['Rate (%)'].mean()) / 100 , 4)
us_rf

In [ ]:
# 포트폴리오 무위험 이자율 = 각 자산군별 국가의 무위험 수익률의 가중치
n_assets = len(tickers)
w_kr = num_kr / n_assets
w_us = num_us / n_assets
p_rf = round((w_kr * kor_rf + w_us * us_rf), 4)
p_rf

# 최소분산 포트폴리오 구성

In [ ]:
# n번개의 랜덤한 비중을 가지는 포트폴리오 구성
p_returns = []
p_volatility = []
p_weights = []

n_assets = len(tickers)
n_ports = 200000

for s in tqdm(range(n_ports)):
  wgt = np.random.random(n_assets)
  wgt /= np.sum(wgt)
  ret = np.dot(wgt, annualReturns)
  vol = np.sqrt(np.dot(wgt.T, np.dot(cov_annual, wgt)))
  p_returns.append(ret)
  p_volatility.append(vol)
  p_weights.append(wgt)

In [ ]:
# 랜덤 포트폴리오 시각화
#plt.figure(figsize=(15, 15))
p_volatility = np.array(p_volatility)
p_returns = np.array(p_returns)
colors = p_returns/p_volatility
plt.style.use('seaborn')
plt.scatter(p_volatility, p_returns, c=colors, marker='o', cmap=mpl.cm.jet)
plt.xlabel('Volatility')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

## 최소분산 최적화 방법1

In [ ]:
# 변동성 최적화 목적함수
def obj_variance(weights, covmat):
    vol = np.sqrt(np.dot(weights, np.dot(covmat, weights.T)))
    return vol

In [ ]:
# 변동성 계산함수
def cal_variance(weights, covmat):
    vol = np.sqrt(np.dot(weights, np.dot(covmat, weights.T)))
    return vol

In [ ]:
# 수익률 계산 함수
def cal_return(weight, annualreturns):
  returns = np.sum(weight * annualreturns)
  return returns

In [ ]:
weights = np.ones([n_assets])/n_assets # 초기 비중은 균일하게 시작
bnds = tuple((0., 1.) for i in range(n_assets)) # 종목당 최소 투자비중 : 0%, 최대 투자가능 비중: 100%
cons = ({'type': 'eq', 'fun': lambda w:  np.sum(w) - 1}) # 투자비중의 합이 1이 되게 설정
res = minimize(obj_variance, weights, cov_annual, method='SLSQP', bounds=bnds, constraints=cons)

In [ ]:
# 최소분산 포트폴리오 시각화
rets = cal_return(res['x'], annualReturns)
vol = cal_variance(res['x'],cov_annual)
p_volatility = np.array(p_volatility)
p_returns = np.array(p_returns)
colors = p_returns/p_volatility
plt.style.use('seaborn')
plt.scatter(p_volatility, p_returns, c=colors, marker='o', cmap=mpl.cm.jet)
plt.scatter(vol, rets, color = 'purple', marker="*", s=300, alpha=1.0)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
print("최적화된 포트폴리오 투자 비중 :", res['x'])
print("최적화된 포트폴리오 투자 수익률 :", cal_return(res['x'], annualReturns))
print("최적화된 포트폴리오 위험정도 :", cal_variance(res['x'],cov_annual))
print("최적화된 포트폴리오 샤프지수정도 :", (rets-p_rf)/vol)
# 극단적인 비중 조정을 하게 된다. 이는 포트폴리오 구성 목적에 맞지 않는다.

## 최소분산 최적화 방법2

In [ ]:
# 랜덤비중 구성 포트폴리오 중 선택
p_returns = []
p_volatility = []
p_weights = []
p_sharp = []
n_assets = len(tickers)
# n_ports = int(input("랜덤하게 만들 포트폴리오 갯수를 입력하세요:"))
n_ports = 200000

for s in tqdm(range(n_ports)):
  wgt = np.random.random(n_assets)
  wgt /= np.sum(wgt)
  ret = np.dot(wgt, annualReturns)
  vol = np.sqrt(np.dot(wgt.T, np.dot(cov_annual, wgt)))
  p_returns.append(ret)
  p_volatility.append(vol)
  p_weights.append(wgt)

port = pd.DataFrame({
    'Returns': p_returns,
    'Volatility': p_volatility,
    'Weights': p_weights,
})
min_vol = port[port['Volatility'] == port['Volatility'].min()]
rets = min_vol['Returns'].iloc[0]
vol = min_vol['Volatility'].iloc[0]

In [ ]:
# 시각화
p_volatility = np.array(p_volatility)
p_returns = np.array(p_returns)
colors = p_returns/p_volatility
plt.style.use('seaborn')
plt.scatter(p_volatility, p_returns, c=colors, marker='o', cmap=mpl.cm.jet)
plt.scatter(vol, rets, color = 'purple', marker="*", s=200, alpha=1.0)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
weights = np.array(min_vol['Weights'].iloc[0])
print("최적화된 포트폴리오 투자 비중(%) :", weights)
print("최적화된 포트폴리오 투자 수익률 :", (min_vol['Returns'].iloc[0]))
print("최적화된 포트폴리오 위험정도 :", (min_vol['Volatility'].iloc[0]))
print("최적화된 포트폴리오 샤프지수정도 :", (rets-p_rf)/vol)
# 투자 수익률이 낮다.

# Sharpe Ratio 최적화

## Sharpe Ratio 최적화방법 1

In [ ]:
# shrpe ratio 최적화 함수 설정
def obj_sharpe(weights, returns, covmat, rf):
    ret = np.dot(weights, returns)
    vol = np.sqrt(np.dot(weights.T, np.dot(covmat, weights)))
    return -(ret-rf)/vol

In [ ]:
# shrpe ratio 계산 함수
def cal_sharpe(weights, returns, covmat, rf):
    ret = np.dot(weights, returns)
    vol = np.sqrt(np.dot(weights.T, np.dot(covmat, weights)))
    return (ret-rf)/vol

In [ ]:
# KOFR = 한국 무위험 수익률 출처:(https://www.kofr.kr/main.jsp)
rf1 = pd.read_csv(f"{PATH}KOFR.csv")
kor_rf = round((rf1['KOFR'].mean()) / 100 , 4)
kor_rf

In [ ]:
# SOFR = 미국 무위험 수익률 출처:(https://www.newyorkfed.org/markets/reference-rates/sofr)
rf2 = pd.read_csv(f"{PATH}SOFR.csv")
us_rf = round((rf2['Rate (%)'].mean()) / 100 , 4)
us_rf

In [ ]:
# 포트폴리오 무위험 이자율 = 각 자산군별 국가의 무위험 수익률의 가중치
n_assets = len(tickers)
w_kr = num_kr / n_assets
w_us = num_us / n_assets
p_rf = round((w_kr * kor_rf + w_us * us_rf), 4)
p_rf

In [ ]:
# sharpe ratio 최적화 방법 1 최적화 함수 사용
n_assets = len(tickers)
weights = np.ones([n_assets])/n_assets
bnds = tuple((0, 1.0) for i in range(n_assets))
cons = ({'type': 'eq', 'fun': lambda w:  np.sum(w) - 1})
res = minimize(obj_sharpe, weights,(annualReturns, cov_annual, p_rf), method='SLSQP', bounds=bnds, constraints=cons)

In [ ]:
print(res['x'])

In [ ]:
# 포트폴리오 시각화
rets = cal_return(res['x'], annualReturns)
vol = cal_variance(res['x'],cov_annual)
p_volatility = np.array(p_volatility)
p_returns = np.array(p_returns)
colors = p_returns/p_volatility
plt.style.use('seaborn')
plt.scatter(p_volatility, p_returns, c=colors, marker='o', cmap=mpl.cm.jet)
plt.scatter(vol, rets, color = 'purple', marker="*", s=200, alpha=1.0)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
weights = res['x']
rounded_weights = [round(element,5)for element in weights]
print("최적화된 포트폴리오 투자 비중(%) :", rounded_weights)
print("최적화된 포트폴리오 투자 수익률 :", cal_return(res['x'], annualReturns))
print("최적화된 포트폴리오 위험정도 :", cal_variance(res['x'],cov_annual))
print("최적화된 포트폴리오 샤프지수정도 :", cal_sharpe(res['x'], annualReturns, cov_annual, p_rf))
# 특정 주식에 비중이 몰려있고 일부 주식은 투자를 안하는 경우가 있다.

## Sharpe Ratio 최적화방법 2

In [ ]:
# 랜덤비중 구성 포트폴리오 중 선택
p_returns = []
p_volatility = []
p_weights = []
p_sharp = []
n_assets = len(tickers)
# n_ports = int(input("랜덤하게 만들 포트폴리오 갯수를 입력하세요:"))
n_ports = 200000

for s in tqdm(range(n_ports)):
  wgt = np.random.random(n_assets)
  wgt /= np.sum(wgt)
  ret = np.dot(wgt, annualReturns)
  vol = np.sqrt(np.dot(wgt.T, np.dot(cov_annual, wgt)))
  sharp = (ret-p_rf)/vol
  p_returns.append(ret)
  p_volatility.append(vol)
  p_weights.append(wgt)
  p_sharp.append(sharp)

port = pd.DataFrame({
    'Returns': p_returns,
    'Volatility': p_volatility,
    'Weights': p_weights,
    'Sharpe Ratio': p_sharp
})

In [ ]:
max_sharpe = port[port['Sharpe Ratio'] == port['Sharpe Ratio'].max()]
rets = max_sharpe['Returns'].iloc[0]
vol = max_sharpe['Volatility'].iloc[0]

In [ ]:
# 시각화
p_volatility = np.array(p_volatility)
p_returns = np.array(p_returns)
colors = p_returns/p_volatility
plt.style.use('seaborn')
plt.scatter(p_volatility, p_returns, c=colors, marker='o', cmap=mpl.cm.jet)
plt.scatter(vol, rets, color = 'purple', marker="*", s=200, alpha=1.0)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
weights = np.array(max_sharpe['Weights'].iloc[0])
rounded_weights = [round(element, 4) for element in weights]
print("최적화된 포트폴리오 투자 비중(%) :", rounded_weights)
print("최적화된 포트폴리오 투자 수익률 :", max_sharpe['Returns'].iloc[0])
print("최적화된 포트폴리오 무위험 수익 대비 수익률 :", round(max_sharpe['Returns'].iloc[0] - p_rf,2))
print("최적화된 포트폴리오 위험정도 :", (max_sharpe['Volatility'].iloc[0]))
print("최적화된 포트폴리오 샤프지수정도 :", (rets-p_rf)/vol)
# 여전히 특정 주식에 대한 비중이 낮다

# risk-parity (포트폴리오 종목들의 위험기여도가 동일하게 구성)

In [ ]:
# 리스크 기여도 계산
def cal_risk_contribution(w, covmat):
    w = np.matrix(w)
    sigma = cal_variance(w, covmat)
    MRC = np.dot(covmat, w.T)
    RC = np.multiply(MRC, w.T) / sigma
    return RC

In [ ]:
def risk_budget_objective(weights, covmat):
    # 포트폴리오 위험계산
    sig_p = obj_variance(weights, covmat)
    # 자산별 위험목표 설정
    risk_target = np.multiply(sig_p, weights)
    # 자산 위험도 계산
    asset_RC = cal_risk_contribution(weights, covmat)
    # 목적함수 평가: 자산 위험 기여금과 각각의 위험 목표 간의 차이의 제곱합
    result = sum(np.square(asset_RC - risk_target.T))[0, 0]
    return result

In [ ]:
n_assets = len(tickers)
weights = np.ones([n_assets]) / n_assets
bnds = tuple((0, 1.0) for i in range(n_assets))
cons = ({'type': 'eq', 'fun': lambda w:  np.sum(w) - 1})
options = {'maxiter': 1000}
res = minimize(risk_budget_objective, weights, cov_annual, method='SLSQP', bounds=bnds, constraints=cons, options = options)

In [ ]:
res['x']

In [ ]:
rets = cal_return(res['x'], annualReturns)
vol = cal_variance(res['x'], cov_annual)
sharp = (rets-kor_rf)/vol
p_volatility = np.array(p_volatility)
p_returns = np.array(p_returns)
colors = p_returns/p_volatility
plt.style.use('seaborn')
plt.scatter(p_volatility, p_returns, c=colors, marker='o', cmap=mpl.cm.jet)
plt.scatter(vol, rets, color = 'purple', marker="*", s=200, alpha=1.0)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
# 자산별 리스크 기여도 시각화
risk_contributions = cal_risk_contribution(res['x'], cov_annual)
risk_contributions_list = [round(float(item), 5) for item in risk_contributions]

plt.figure(figsize=(10, 5))
plt.bar(range(len(risk_contributions_list)), risk_contributions_list,tick_label = adjClose.columns, align='center')
plt.xlabel('Asset')
plt.ylabel('Risk Contribution')
plt.title('Risk Contributions of Assets in Risk Parity Portfolio')
plt.show()

In [ ]:
print("최적화된 포트폴리오 투자 비중(%) :", [round(element,4) for element in res['x']])
print("최적화된 포트폴리오 투자 수익률 :" + str(cal_return(res['x'], annualReturns)))
print("최적화된 포트폴리오 위험정도 :", cal_variance(res['x'],cov_annual))
print("최적화된 포트폴리오 샤프지수정도 :", cal_sharpe(res['x'], annualReturns, cov_annual, p_rf))
print("최적화된 포트폴리오 자산별 리스크 기여도정도 :", risk_contributions_list)
# 비록 샤프비율 최적화에 비해 투자 수익률이 낮게 나오지만 포트폴리오 투자 비중이 극단적이지 않음으로 포트폴리오 구성 목적에 더 적합하다고 생각한다.

# 테스트

In [ ]:
# 학습 데이터셋 기간 설정
start_1 = datetime.datetime(2023, 8, 28)
end_1 = datetime.datetime(2023, 11, 16)

In [ ]:
num_kr = pd.Series(tickers).str.endswith(('.KS', '.KQ')).sum()
num_us = (~pd.Series(tickers).str.endswith(('.KS', '.KQ'))).sum()
adjClose = pd.DataFrame()
exch_list =['.KS','.KQ']

for item in tickers:
        data = web.get_data_yahoo(item, start=start_1, end=end_1)
        if len(data)<1:
            a, b = item.split('.')
            if b == 'KS':
                item = a + '.KQ'
                data = web.get_data_yahoo(item, start=start_1, end=end_1)
                adjClose[item] = data['Adj Close'].astype(float)
            if b == 'KQ':
                item = a + '.KS'
                data = web.get_data_yahoo(item, start=start_1, end=end_1)
                adjClose[item] = data['Adj Close'].astype(float)

        adjClose[item] = data['Adj Close'].astype(float)

adjClose.isnull().sum()
# 로그 리턴 계산
daily_log_returns = np.log(adjClose/adjClose.shift(1)).dropna()
# 연간 로그 수익률 계산
meanReturns = daily_log_returns.mean()
annualReturns = daily_log_returns.mean() * 50
# annualReturns = daily_log_returns.mean() * 250
# 종목간의 수익률 공분산 계산
cov_daily = daily_log_returns.cov()
cov_annual = cov_daily * 50
# cov_annual = cov_daily * 250

In [ ]:
res['x']

In [ ]:
p_rets = cal_return(res['x'], annualReturns)
p_vol = cal_variance(res['x'], cov_annual)
p_sharp = (p_rets-p_rf)/p_vol
weights = np.ones([n_assets]) / n_assets
i_rets = cal_return(weights, annualReturns)
i_vol = cal_variance(weights, cov_annual)
i_sharp = (i_rets-p_rf)/i_vol

In [ ]:
print(p_rets)
print(p_vol)
print(p_sharp)
print(i_rets)
print(i_vol)
print(i_sharp)

In [ ]:
kospi =  web.get_data_yahoo('^KS11', start=start_1, end=end_1)
kospi = kospi['Adj Close'].round().astype(int)
dailySimpleReturns_kospi = kospi.pct_change().dropna()
daily_log_returns_kospi = np.log(kospi/kospi.shift(1)).dropna()
# annualise daily returns. 252 trading days in a year
meanReturns_kospi = np.matrix(daily_log_returns_kospi.mean())
annualReturns_kospi = daily_log_returns_kospi.mean() * 50
annualReturns_kospi

In [ ]:
kospi =  web.get_data_yahoo('^KQ11', start=start_1, end=end_1)
kospi = kospi['Adj Close'].round().astype(int)
dailySimpleReturns_kospi = kospi.pct_change().dropna()
daily_log_returns_kospi = np.log(kospi/kospi.shift(1)).dropna()
# annualise daily returns. 252 trading days in a year
meanReturns_kospi = np.matrix(daily_log_returns_kospi.mean())
annualReturns_kospi = daily_log_returns_kospi.mean() * 50
annualReturns_kospi

In [ ]:
kospi =  web.get_data_yahoo('^IXIC', start=start_1, end=end_1)
kospi = kospi['Adj Close'].round().astype(int)
dailySimpleReturns_kospi = kospi.pct_change().dropna()
daily_log_returns_kospi = np.log(kospi/kospi.shift(1)).dropna()
# annualise daily returns. 252 trading days in a year
meanReturns_kospi = np.matrix(daily_log_returns_kospi.mean())
annualReturns_kospi = daily_log_returns_kospi.mean() * 50
annualReturns_kospi

In [ ]:
kospi =  web.get_data_yahoo('^GSPC', start=start_1, end=end_1)
kospi = kospi['Adj Close'].round().astype(int)
dailySimpleReturns_kospi = kospi.pct_change().dropna()
daily_log_returns_kospi = np.log(kospi/kospi.shift(1)).dropna()
# annualise daily returns. 252 trading days in a year
meanReturns_kospi = np.matrix(daily_log_returns_kospi.mean())
annualReturns_kospi = daily_log_returns_kospi.mean() * 50
annualReturns_kospi

# 포트폴리오 비교평가

## 랜덤추출

In [ ]:
import random
import statistics

In [ ]:
def get_tickername(df):
    if len(df) != 0:
        res = []
        for data in df:
            ticker,exch = data.split()
            if exch ==' ':
                res.append(ticker)
            else:
                res.append(ticker+'.'+exch)
        return res

    else:
        print("해당 종목은 찾을 수 없는 종목입니다")
        return None

In [ ]:
stocklist = list(relation_score['ticker'])
selected_tickers = random.sample(stocklist, 10)
print(selected_tickers)

In [ ]:
num_kr = pd.Series(tickers).str.endswith(('.KS', '.KQ')).sum()
num_us = (~pd.Series(tickers).str.endswith(('.KS', '.KQ'))).sum()
exch_list =['.KS','.KQ']
stocklist = list(relation_score['ticker'])
test_return_list = []
test_vol_list = []
test_sharpe_list = []
error = True

# while error:
for x in selected_tickers:
    tickers = get_portfolio(x,relation_score)
    count = 0
    adjClose = pd.DataFrame()
    for item in tickers:
        data = web.get_data_yahoo(item, start=start_1, end=end_1)
        if len(data)<1:
            a, b = item.split('.')
            if b == 'KS':
                item = a + '.KQ'
                data = web.get_data_yahoo(item, start=start_1, end=end_1)
                adjClose[item] = data['Adj Close'].astype(float)
            if b == 'KQ':
                item = a + '.KS'
                data = web.get_data_yahoo(item, start=start_1, end=end_1)
                adjClose[item] = data['Adj Close'].astype(float)

        adjClose[item] = data['Adj Close'].astype(float)

    # 로그 리턴 계산
    daily_log_returns = np.log(adjClose/adjClose.shift(1)).dropna()
    # 연간 로그 수익률 계산
    meanReturns = daily_log_returns.mean()
    annualReturns = daily_log_returns.mean() * 250
    # 종목간의 수익률 공분산 계산
    cov_daily = daily_log_returns.cov()
    cov_annual = cov_daily * 250
    n_assets = len(tickers)
    weights = np.ones([n_assets]) / n_assets
    bnds = tuple((0, 1.0) for i in range(n_assets))
    cons = ({'type': 'eq', 'fun': lambda w:  np.sum(w) - 1})
    options = {'maxiter': 1000}
    res = minimize(risk_budget_objective, weights, cov_annual, method='SLSQP', bounds=bnds, constraints=cons, options = options)
    test_return_list.append(cal_return(res['x'], annualReturns))
    test_vol_list.append(cal_variance(res['x'], cov_annual))
    test_sharpe_list.append(cal_sharpe(res['x'], annualReturns, cov_annual, p_rf))
    # if pd.isna(cal_variance(res['x'], cov_annual)):
    #     selected_tickers[count] = random.sample(stocklist, 1)[0]
    print(x,"의최적화된 포트폴리오 투자 비중(%) :", [round(element,4) for element in res['x']])
    print(x,"의최적화된 포트폴리오 투자 수익률 :" + str(cal_return(res['x'], annualReturns)))
    print(x,"의최적화된 포트폴리오 위험정도 :", cal_variance(res['x'],cov_annual))
    print(x,"의최적화된 포트폴리오 샤프지수정도 :", cal_sharpe(res['x'], annualReturns, cov_annual, p_rf))
    print(x,"의최적화된 포트폴리오 자산별 리스크 기여도정도 :", risk_contributions_list)

#         if not pd.isna(cal_variance(res['x'], cov_annual)):
#             error = False








In [ ]:
print(statistics.mean(test_return_list))
print(statistics.mean(test_vol_list))
print(statistics.mean(test_sharpe_list))

## 상관관계 비중 조정 추출

In [ ]:
def cal_port_ratio_risk(tickers):
    num_kr = pd.Series(tickers).str.endswith(('.KS', '.KQ')).sum()
    num_us = (~pd.Series(tickers).str.endswith(('.KS', '.KQ'))).sum()
    adjClose = pd.DataFrame()
    #exch_list =['.KS','.KQ']

    for item in tickers:
            data = web.get_data_yahoo(item, start=start, end=end)
            if len(data)<1:
                a, b = item.split('.')
                if b == 'KS':
                    item = a + '.KQ'
                    data = web.get_data_yahoo(item, start=start, end=end)
                    adjClose[item] = data['Adj Close'].astype(float)
                if b == 'KQ':
                    item = a + '.KS'
                    data = web.get_data_yahoo(item, start=start, end=end)
                    adjClose[item] = data['Adj Close'].astype(float)

            adjClose[item] = data['Adj Close'].astype(float)

    # 로그 리턴 계산
    daily_log_returns = np.log(adjClose/adjClose.shift(1)).dropna()
    # 연간 로그 수익률 계산
    meanReturns = daily_log_returns.mean()
    annualReturns = daily_log_returns.mean() * 252
    # 종목간의 수익률 공분산 계산
    cov_daily = daily_log_returns.cov()
    cov_annual = cov_daily * 252

    n_assets = len(tickers)
    weights = np.ones([n_assets]) / n_assets
    bnds = tuple((0, 1.0) for i in range(n_assets))
    cons = ({'type': 'eq', 'fun': lambda w:  np.sum(w) - 1})
    options = {'maxiter': 1000}
    res = minimize(risk_budget_objective, weights, cov_annual, method='SLSQP', bounds=bnds, constraints=cons, options = options)

    print("최적화된 포트폴리오 투자 비중(%) :", [round(element,4) for element in res['x']])
    print("최적화된 포트폴리오 투자 수익률 :" + str(cal_return(res['x'], annualReturns)))
    print("최적화된 포트폴리오 위험정도 :", cal_variance(res['x'],cov_annual))
    print("최적화된 포트폴리오 샤프지수정도 :", cal_sharpe(res['x'], annualReturns, cov_annual, p_rf))
    print("최적화된 포트폴리오 자산별 리스크 기여도정도 :", risk_contributions_list)
    # 비록 샤프비율 최적화에 비해 투자 비중이 낮게 나오지만 포트폴리오 투자 비중이 극단적이지 않음으로 포트폴리오 구성 목적에 더 적합하다고 생각한다.


In [ ]:
# 상관관계 정도 상위 9개 종목
tickers = get_portfolio_ratio('AMZN',relation_score, 9, 0)
cal_port_ratio_risk(tickers)

In [ ]:
# 상관관계 정도 하위 9개 종목
tickers = get_portfolio_ratio('AMZN',relation_score, 0, 9)
cal_port_ratio_risk(tickers)

In [ ]:
# 상관관계 정도 상위 5개 하위 4개 종목
tickers = get_portfolio_ratio('AMZN',relation_score, 5, 4)
cal_port_ratio_risk(tickers)

In [ ]:
# 상관관계 정도 상위 4개 하위 5개 종목
tickers = get_portfolio_ratio('AMZN',relation_score, 4, 5)
cal_port_ratio_risk(tickers)

In [ ]:
tickers

In [ ]:
data = web.get_data_yahoo('AMZN', start=start, end=end)
adjClose = pd.DataFrame()
adjClose['Adj Close'] = data['Adj Close'].astype(float)

# 로그 리턴 계산
daily_log_returns = np.log(adjClose/adjClose.shift(1)).dropna()
# 연간 로그 수익률 계산
meanReturns = daily_log_returns.mean()
annualReturns = daily_log_returns.mean() * 252
# 종목간의 수익률 공분산 계산
sigma = daily_log_returns.std()
annual_sigma = sigma * np.sqrt(252)

In [ ]:
print(annualReturns)
print(annual_sigma)
print((annualReturns - us_rf) / annual_sigma)